In [4]:
import pandas as pd
import requests
import json
import numpy as np


In [5]:
filepath = """../2019 Food Access Research Atlas Data/Food Access Research Atlas.csv"""
data = pd.read_csv(filepath)

In [6]:
texas = data.loc[data["State"]=="Texas",:]
texas = texas[['CensusTract','State','County','Urban','Pop2010','OHU2010','LILATracts_1And10','LILATracts_halfAnd10','HUNVFlag','LowIncomeTracts',
'PovertyRate','MedianFamilyIncome','LA1and10','LAhalfand10','LATracts_half','LALOWI1_10','LALOWI05_10','lahunvhalf','lahunvhalfshare']]

In [7]:
# Open McDonald's JSON file
json_file = open("mcdata.json")
mcdonalds = json.load(json_file)["features"]

# Store all the McDonalds in Texas
mcdonalds_tx = [store for store in mcdonalds if store['properties']['state']=='TX']

# Store lats and lons of each McDonald's restaurant in US
lats = [store['geometry']['coordinates'][1] for store in mcdonalds_tx]
lons = [store['geometry']['coordinates'][0] for store in mcdonalds_tx]

In [8]:
# Store the base URL - this API takes geographical coordinates and returns the Census Tract that those coordinates fall into
base_url = "https://geo.fcc.gov/api/census/area?"

# Initialize an empty list to store the census tract info for each store
census_tracts = []

# Loop through the stores in texas using enumerate
for i in range(len(mcdonalds_tx)):

    # create a url for each store using the latitude and longitutde and base
    store_url = f'{base_url}lat={lats[i]}&lon={lons[i]}&censusYear=2010&format=json'

   # Run API request for each store
    try:
        # Store the results
       results = requests.get(store_url).json()
       
       # Store the tract number - NOTE: only the first 11 digits because the API number is more specific than the Food Access Research Atlas
       tract = results["results"][0]["block_fips"][0:-4]

        # Append the tract number to a list
       census_tracts.append(tract)

    except:
       print("No tract available")
       pass


In [9]:
census_tracts = [int(tract) for tract in census_tracts]
mctracts = pd.DataFrame(census_tracts, columns=["CensusTract"])
mctracts["HasMcD"] = 1
mctracts_grp = mctracts.groupby("CensusTract")
mctracts2 = pd.DataFrame(mctracts.groupby("CensusTract")["HasMcD"].sum())
mctracts = mctracts.drop_duplicates("CensusTract")
mctracts_final = pd.merge(mctracts,mctracts2,on="CensusTract")
mctracts_final = mctracts_final.rename(columns = {"HasMcD_x":"HasMcD","HasMcD_y":"CountMcD"})


In [11]:
tx_tracts = texas.merge(mctracts_final,how="left", on="CensusTract")
tx_tracts[["HasMcD","CountMcD"]] = tx_tracts[["HasMcD","CountMcD"]].fillna(0)
tx_tracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5238 entries, 0 to 5237
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CensusTract           5238 non-null   int64  
 1   State                 5238 non-null   object 
 2   County                5238 non-null   object 
 3   Urban                 5238 non-null   int64  
 4   Pop2010               5238 non-null   int64  
 5   OHU2010               5238 non-null   int64  
 6   LILATracts_1And10     5238 non-null   int64  
 7   LILATracts_halfAnd10  5238 non-null   int64  
 8   HUNVFlag              5238 non-null   int64  
 9   LowIncomeTracts       5238 non-null   int64  
 10  PovertyRate           5238 non-null   float64
 11  MedianFamilyIncome    5187 non-null   float64
 12  LA1and10              5238 non-null   int64  
 13  LAhalfand10           5238 non-null   int64  
 14  LATracts_half         5238 non-null   int64  
 15  LALOWI1_10           